In [8]:
import os
import pandas as pd
import numpy as np

carpeta_tweets = "./tweets"

categorias_posibles = ["Violencia", "Homofobia", "Xenofobia", "Incensurable"]

dataframes_por_archivo = {}

for nombre_archivo in os.listdir(carpeta_tweets):
    if nombre_archivo.endswith(".txt"):
        ruta_archivo = os.path.join(carpeta_tweets, nombre_archivo)

        datos = []
        with open(ruta_archivo, mode='r', encoding='utf-8') as archivo:
            for linea in archivo:
                partes = linea.strip().split("#############")
                if len(partes) != 3:
                    continue

                texto, etiquetas, tweet_padre = partes

                etiquetas_set = set(et.strip() for et in etiquetas.split(",") if et.strip())
                if not etiquetas_set:
                    etiquetas_set = {"Incensurable"}

                fila = {
                    "Tweet": texto,
                    "Tweet_Padre": tweet_padre
                }

                for cat in categorias_posibles:
                    fila[cat] = int(cat in etiquetas_set)

                datos.append(fila)

        nombre_df = os.path.splitext(nombre_archivo)[0]
        df = pd.DataFrame(datos)
        dataframes_por_archivo[nombre_df] = df

for nombre, df in dataframes_por_archivo.items():
    print(f"Primeras filas de {nombre}:")
    print(df.head())
    print("\n---\n")


Primeras filas de TweetsLako:
                                               Tweet    Tweet_Padre  \
0                   No queremos más venecos en Chile  SinTweetPadre   
1  El negro Piñera se murió hace dos meses y ya N...  SinTweetPadre   
2  “De venir de la calle …a vivir de lujos gracia...  SinTweetPadre   
3  5:47 y ya está oscuro, ¿esa wea querian los in...  SinTweetPadre   
4  Uno se pregunta como negros qls pobres q vagan...  SinTweetPadre   

   Violencia  Homofobia  Xenofobia  Incensurable  
0          0          0          1             0  
1          0          0          0             1  
2          0          0          0             1  
3          0          0          0             1  
4          0          0          1             0  

---



In [9]:
for nombre, df in dataframes_por_archivo.items():
    print(f"Conteo por categoría para: {nombre}")
    conteo_por_categoria = df[categorias_posibles].sum().sort_values(ascending=False)
    print(conteo_por_categoria)
    print("\n---\n")



Conteo por categoría para: TweetsLako
Violencia       50
Homofobia       50
Xenofobia       50
Incensurable    50
dtype: int64

---



In [10]:
df_unificado = pd.concat(dataframes_por_archivo.values(), ignore_index=True)
df_unificado.head()

,Tweet,Tweet_Padre,Violencia,Homofobia,Xenofobia,Incensurable
0,No queremos más venecos en Chile,SinTweetPadre,0,0,1,0
1,El negro Piñera se murió hace dos meses y ya N...,SinTweetPadre,0,0,0,1
2,“De venir de la calle …a vivir de lujos gracia...,SinTweetPadre,0,0,0,1
3,"5:47 y ya está oscuro, ¿esa wea querian los in...",SinTweetPadre,0,0,0,1
4,Uno se pregunta como negros qls pobres q vagan...,SinTweetPadre,0,0,1,0


In [11]:
tweets_duplicados = df_unificado[df_unificado.duplicated(subset='Tweet', keep=False)]

print("Tweets duplicados:")
print(tweets_duplicados)


Tweets duplicados:
Empty DataFrame
Columns: [Tweet, Tweet_Padre, Violencia, Homofobia, Xenofobia, Incensurable]
Index: []


In [12]:
df_unificado['ID'] = range(1, len(df_unificado) + 1)

columnas_ordenadas = ['ID'] + [col for col in df_unificado.columns if col != 'ID']
df_unificado = df_unificado[columnas_ordenadas]

df_unificado.head()


,ID,Tweet,Tweet_Padre,Violencia,Homofobia,Xenofobia,Incensurable
0,1,No queremos más venecos en Chile,SinTweetPadre,0,0,1,0
1,2,El negro Piñera se murió hace dos meses y ya N...,SinTweetPadre,0,0,0,1
2,3,“De venir de la calle …a vivir de lujos gracia...,SinTweetPadre,0,0,0,1
3,4,"5:47 y ya está oscuro, ¿esa wea querian los in...",SinTweetPadre,0,0,0,1
4,5,Uno se pregunta como negros qls pobres q vagan...,SinTweetPadre,0,0,1,0


In [15]:
texto_a_id = dict(zip(df_unificado['Tweet'], df_unificado['ID']))

def obtener_id_padre(texto_padre):
    if texto_padre == "SinTweetPadre":
        return np.nan
    return texto_a_id.get(texto_padre, "X")

df_unificado['ID_Tweet_Padre'] = df_unificado['Tweet_Padre'].apply(obtener_id_padre)

num_casos_X = (df_unificado['ID_Tweet_Padre'] == "X").sum()
print(f"Número de tweets con 'X' en ID_Tweet_Padre (no se encontró el tweet padre): {num_casos_X}")

df_unificado.head()


Número de tweets con 'X' en ID_Tweet_Padre (no se encontró el tweet padre): 92


,ID,Tweet,Tweet_Padre,Violencia,Homofobia,Xenofobia,Incensurable,ID_Tweet_Padre
0,1,No queremos más venecos en Chile,SinTweetPadre,0,0,1,0,NaN
1,2,El negro Piñera se murió hace dos meses y ya N...,SinTweetPadre,0,0,0,1,NaN
2,3,“De venir de la calle …a vivir de lujos gracia...,SinTweetPadre,0,0,0,1,NaN
3,4,"5:47 y ya está oscuro, ¿esa wea querian los in...",SinTweetPadre,0,0,0,1,NaN
4,5,Uno se pregunta como negros qls pobres q vagan...,SinTweetPadre,0,0,1,0,NaN
